In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import level2_roi_extractor
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

In [3]:
# #filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
# sst_roi_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|finger|motor|response inhibition')]

# ## SST
# train_betas_with_data = get_data_for_confirmed_train_subjs(
#     beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
#     nonbids_data_path = config['nonbids_data_path'],
#     #ml_data_folderpath = ml_data_folderpath,
#     ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
#     dropbox_datapath=config['dropbox_data_dir'],
#     exclude_test_subjs=False
# )
# train_betas_with_data['wave']=1

# #we're not interestd in getting contrasts; comment this out.
# #betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
# betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])


### W1 W2 analysis

In [4]:
#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/',
    config['mask_location'] + 'striatum/',
    config['mask_location']
]
wave_compare_roi_df = get_mask_df_from_mask_locations(mask_locations)

In [5]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = wave_compare_roi_df.loc[wave_compare_roi_df['mask_label'].str.contains('response inhibition|striatum_joint_reward|value')]


In [6]:

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)

train_betas_with_data['wave']='all'

loaded 275 rows from data_by_ppt.csv
255 subjects with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
7 subjects excludeable for missing scan data.
251 subjects remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
217 subjects remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 250
beta paths after exclusion via nsc_subject_exclusions and the provision useable_dev_id list: 185


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_betas_with_data.sort_values('subject_id', inplace=True)
/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_78576/377843695.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_betas_with_data['wave']='all'


In [7]:
contrasts_with_paths = get_contrasts_for_betas(train_betas_with_data)

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV008/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_cond

### Test code

In [9]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)'
    ]

l2_roi_extractor = level2_roi_extractor(center_data=True,scale_data=True)
l2_roi_extractor.load_all_images_simultaneously=True

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)

response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 185images...
...loaded.
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016', 'DEV017', 'DEV018', 'DEV019', 'DEV020', 'DEV021', 'DEV023', 'DEV024', 'DEV026', 'DEV028', 'DEV029', 'DEV030', 'DEV031', 'DEV033', 'DEV036', 'DEV038', 'DEV040', 'DEV041', 'DEV042', 'DEV043', 'DEV044', 'DEV046', 'DEV047', 'DEV048', 'DEV049', 'DEV051', 'DEV052', 'DEV053', 'DEV054', 'DEV055', 'DEV056', 'DEV057', 'DEV058', 'DEV059', 'DEV060', 'DEV062', 'DEV063', 'DEV065', 'DEV066', 'DEV067', 'DEV068', 'DEV070', 'DEV071', 'DEV072', 'DEV073', 'DEV074', 'DEV075', 'DEV076', 'DEV077', 'DEV078', 'DEV079', 'DEV081', 'DEV083', 'DEV084', 'DEV085', 'DEV086', 'DEV087', 'DEV08

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.01383226975097848
activity scalar is [ 0.36261657  1.3819886   0.7864166  -0.22164182  0.2861319   0.16053458
 -0.16205356  0.23331296 -0.48423204  1.416872    0.27373323 -0.73542815
 -0.01585262 -0.12822649  0.02403402 -0.02044605  0.08577375 -1.0108622
 -0.38104394  0.0697014  -0.02671816 -0.2886582  -0.3200187  -0.31837144
  0.54880404  0.31967318 -0.66203445  0.1471083  -0.2343194  -0.14945753
 -0.04388342  0.29343837 -0.2296284  -0.27701122 -0.05340985  0.03318328
  0.05451996  0.83666384 -0.2718973  -0.163705   -0.00264718  0.07892192
 -0.1511486   0.3881973   0.6961355   0.07050441 -0.19135454 -0.01768012
 -0.00929692 -0.10734785 -0.2976709  -0.10732881  0.19313154 -0.16653524
 -0.24638508 -0.5031345   0.04280037  0.195424   -0.5456205   0.29902697
  0.30222267  0.22353657  1.0327835   0.93392915 -0.09857618 -0.105351
 -0.12789279  0.38240084  0.3338696   0.15436454 -0.09109061 -0.4815001
 -0.7835805  -0.01147626 -0.38299364 -0.4738493  -0.6283568  

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.016650108360635286
activity scalar is [-0.14479417  1.622632    0.6203338  -0.22370344  0.23056398 -0.3535745
 -0.11454533 -0.07059941 -0.44756663  1.7713755  -0.11939142 -0.39414677
  0.06197784  0.22134025 -0.40582845  0.04569378  0.04257524 -0.8650495
 -0.2941219  -0.01480182  0.12717319 -0.4648272  -0.41754395 -0.48098174
  0.15166502  0.33163434 -0.54229856  0.07694013 -0.11679337  0.04716239
 -0.4265601   0.30456427 -0.5200961  -0.1725334  -0.13718557  0.00477735
  0.14600421  0.6912819  -0.22375679 -0.6104566  -0.03560268  0.26633275
  0.17484142  0.23565508  0.24672543 -0.09730398 -0.44902843 -0.20443295
  0.06394731  0.16864748 -0.32883322 -0.00994022  0.26716796 -0.08369117
  0.29141536 -0.39769402  0.19278327  0.33392984 -0.5836312   0.69221777
  0.3187483   0.39735156  1.0103697   0.35939994 -0.08963076 -0.34647265
  0.15794829  0.55658275  0.47239342  0.29038563 -0.03851789 -0.51673
 -0.9488108  -0.00895145 -0.23177902 -0.1218491  -0.19264601 

...

proportion of image active: 0.016650108360635286
activity scalar is [ 0.0427396   0.12018088 -0.6455735   0.06457185 -0.38942733 -0.08374206
  0.25100216  0.16585423 -0.2318489   0.32482743  0.09418131  0.28199133
 -0.13056764 -0.12834114  0.20901093  0.12848623  0.14109012  0.5665
 -0.16822164 -0.53887874 -0.00303569  0.19986235  0.10222505  0.1597235
 -0.01914659  0.0144631   0.48818684 -0.19976215  0.0638254   0.5946688
 -0.05968986 -0.12760371 -0.15097724 -0.47923297 -0.13028924  0.2212508
 -0.38266897 -0.04591552  0.19733693 -0.12821767 -0.36822402  0.37105516
  0.04097809  0.6681977   0.04649857 -0.41028595  0.0844656   0.21050273
  0.15698938 -0.1513417   0.2511543   0.01957365 -0.00280715 -0.19071849
 -0.6899244   0.23171216  0.14633279 -0.6419756  -0.39364284  0.274435
 -0.03847651 -0.49022195 -0.43696213 -1.5982698  -0.02141589 -0.3735483
  0.07369242 -0.11823762  0.23072037 -0.10540941  0.68061763  0.45320755
 -0.59566855  0.24015112 -0.06030255  0.27915794  1.0721365   0.12

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.005345483279191523
activity scalar is [ 0.10086448  0.16559523 -0.3114966  -0.00381245 -0.30738875 -0.09527451
  0.38286927 -0.02449302 -0.18484141  0.6251325   0.3676715   0.5266768
 -0.01344559 -0.16138318  0.03278181 -0.45827433  0.15431696  0.5940529
 -0.23912376 -0.61255217  0.22420909 -0.03787821  0.42489198  0.45104298
 -0.28721234 -0.09780402  0.44414955 -0.06581522 -0.00634528  0.7126218
  0.03133508 -0.40714654 -0.19171624 -0.42846242 -0.23913787  0.39596063
 -0.55875707  0.13169172  0.18222132 -0.7032911  -0.18316378  0.30340782
 -0.24827331  1.1785216  -0.09136644  0.14571452  0.14243934 -0.0421092
 -0.44005254  0.06768133  0.07668135  0.2685631  -0.04657733 -0.11833917
 -0.3616335  -0.19374686  0.21689163 -0.33675212 -0.5252446  -0.08811191
  0.01379382 -0.32778782 -0.29378882 -1.8494581  -0.06120876 -0.08715864
  0.11604671 -0.3483775  -0.11130153  0.08257754  0.6624242   0.5462505
 -0.5113318   0.02152859 -0.46845773  0.3465975   1.0251883  

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.0007864810288022106
activity scalar is [ 0.7850387   0.61537224 -1.8063442   0.65289754 -0.2853717  -0.03666242
  0.24704306  0.5152793  -0.35333988 -0.19339345  0.3768356  -0.32785985
 -0.65790784 -0.258551    0.2228578   0.47414312  0.44657424  0.80739105
 -0.06559662 -0.17708468 -0.03881057  0.1634784  -0.84570616  0.45556253
 -0.05648656  0.14106059  0.07396821 -0.25031245  0.15998805  0.6696586
  0.37750617  0.03227441 -0.11528522 -0.82737076 -0.6429741  -0.16374588
 -0.09071553 -0.29771882  0.44723463  0.6254917  -0.7078075   0.43992904
  0.22989397  0.74454105  0.01098379 -1.6685944  -0.25598866  0.63458794
  0.8350009  -0.8116265   1.1457201  -0.18192057  0.57801414 -0.18031435
 -1.646478    1.2894425   0.10528048 -1.6671526  -0.8340139   0.13488656
 -0.5956517  -1.393093   -0.3845935  -1.5344895  -0.2507667  -1.0312724
 -0.13625798  0.19871877  0.8872261  -0.08791593  1.0860928   0.75818574
 -0.8467322   0.58776754  0.33549204  0.33781278  1.56995

In [10]:
roi_data_sst_conditions.to_csv(config['dropbox_data_dir'] + '/subject_sst_health_avg_roi_data_raw_zscored.csv')

### spatial normalization

In [12]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)'
    ]

l2_roi_extractor = level2_roi_extractor(center_data=True,scale_data=True)
l2_roi_extractor.load_all_images_simultaneously=True
l2_roi_extractor.image_standardize=True
l2_roi_extractor.brain_mask_path = config['mask_location'] + 'aim3/masks/neurosynth/brain_mask.nii.gz'

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)

response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 185images...


KeyboardInterrupt: 

# TEMP

In [13]:
import os
import nilearn as nil
import pandas as pd
import numpy as np
from direct_regression.get_all_series import get_beta_img, mask_3d_subject_image, mask_4d_subject_image, signature_weight_3d_subject_image

#for backwards compatibility, we expose some functions that work without the class

def get_roi_data_for_beta(
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned):
    return level2_roi_extractor().get_roi_data_for_beta(
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned)

def get_roi_data_for_l2_betas():
    return level2_roi_extractor().get_roi_data_for_l2_betas()

def load_rois(mask_df):
    return level2_roi_extractor().load_rois(mask_df)

def get_roi_data_for_multirun_l2_betas(beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False):
    return level2_roi_extractor().get_roi_data_for_multirun_l2_betas(beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False)

class level2_roi_extractor:

    # define init
    def __init__(self,center_data=False, scale_data=False):
        self.center_data = center_data
        self.scale_data = scale_data
        self.load_all_images_simultaneously = False
        self.image_standardize = False
        self.brain_mask_path = None


    def load_rois(self, mask_df):
        raw_mask_list = []
        for m_i, m_set in mask_df.iterrows():
            print(m_set['mask_label'])
            print('producing the vector for this mask/series...')
            #get the mask
            mask_raw = nil.image.load_img(m_set['mask_path'])
            raw_mask_list = raw_mask_list + [mask_raw]

        return raw_mask_list
    
    def mask_subject_image(self,roi_raw, active_img_cleaned, bin_threshold):
        if len(active_img_cleaned.shape)==3 or active_img_cleaned.shape[3]==1:
            masked_img = mask_3d_subject_image(roi_raw, active_img_cleaned, bin_threshold)
        elif len(active_img_cleaned.shape)==4:
            masked_img = mask_4d_subject_image(roi_raw, active_img_cleaned, bin_threshold)
        return(masked_img)
    

    def get_roi_data_for_beta(self, 
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned):
        roi_data_for_beta = []
        roi_df = roi_df.copy().reset_index(drop=True)
        #rules for input: 
        # length of active_img_cleaned can be 1 or more
        # subject_id should be a scalar, a string, or an iterable with the same number of images in active_img_cleaned
        assert (len(subject_id)==active_img_cleaned.shape[3] or isinstance(subject_id,str) or (not hasattr(subject_id, "__iter__")))
        # length of beta_name should also be 1 or the same as the number of images in active_img_cleaned
        assert (len(beta_name)==active_img_cleaned.shape[3] or isinstance(beta_name,str) or (not hasattr(beta_name, "__iter__")))
        #if subject_id or wave are iterables, then beta_name should be, too
        if (
            (hasattr(subject_id, "__iter__") and subject_id is not str) or
            (hasattr(wave, "__iter__") and wave is not str)):
            assert (hasattr(beta_name, "__iter__") and beta_name is not str)
        # masks are always independent of the number of images in active_img_cleaned
        
        for m_i, m_set in roi_df.iterrows():
            print(str(subject_id) + ", " + str(wave) + ", " + str(condition) + ", " + m_set['mask_label'])
            
            roi_raw = raw_roi_list[m_i]

            if 'image_type' not in m_set.keys():
                #no image type set. need to handle this case first in order to avoid errors.
                active_img_masked = self.mask_subject_image(roi_raw, active_img_cleaned, bin_threshold=np.max(roi_raw.get_fdata())/1000)
            elif m_set['image_type'] == 'mask':
                #binary threshold has to be not quite zero because, with the dtype transform, 
                # some zeros get rounded up to a very-close-to-zero amount.
                active_img_masked = self.mask_subject_image(roi_raw, active_img_cleaned, bin_threshold=np.max(roi_raw.get_fdata())/1000)
            elif m_set['image_type'] == 'signature':
                #handles a bit differently; for signatures we don't set a binary threshold; the images are simply multiplied together.
                active_img_masked = signature_weight_3d_subject_image(roi_raw, active_img_cleaned)
            else:
                raise ValueError("image type not recognized")
            
            if len(active_img_masked.shape)==1:
                activity_scalar = active_img_masked.mean()
            elif len(active_img_masked.shape)==2:
                #get row means of the masked image
                activity_scalar = active_img_masked.mean(axis=1)


            print("activity scalar is " + str(activity_scalar))
            roi_data_for_beta.append(pd.DataFrame({
                'subject_id': subject_id,#r['subject_id'],
                'wave':wave,# r['wave'],
                'spm_l2_path':spm_l2_path,#r['spm_l2_path'],
                'condition' : condition,
                'beta_name': beta_name,#r[colname],
                'mask_label': m_set['mask_label'],
                'roi_activity': activity_scalar
            }))
        return roi_data_for_beta



    def get_roi_data_for_multirun_l2_betas(self, beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False):
        ###
        #takes a beta list where each entry is one beta image 
        #mean-centers each mask
        #and gets the ROI data for each beta image
        
        
        raw_mask_list = load_rois(mask_df)

        roi_data_all = []

        #iterate through each subject and wave
        for sub_i,sub in enumerate(np.unique(beta_list.subject_id)):
            #and through each condition
            for wave_i,wave in enumerate(np.unique(beta_list.wave)):
                #get the conditions applicable for this subject
                session_betas = beta_list[(beta_list['subject_id']==sub) & (beta_list['wave']==wave)]
                #iterate through each condition
                for condition in set(np.unique(session_betas.beta_description)).intersection(set(condition_list)):
                    #get the betas for this condition
                    condition_betas = session_betas[session_betas['beta_description']==condition]


                    #iterate through each beta
                    for i, r in condition_betas.iterrows():
                        #some checks where skipping is necessary
                        if type(r['beta_fname'])==float:
                            if np.isnan(r['beta_fname']):
                                continue
                            #we want an error thrown if somehow it's a float but not nan,
                            # so we handle that case here.
                        if r['beta_fname'] is None:
                            continue
                        
                    
                        #get the beta image from which to pull an ROI
                        beta_img_filepath = r['spm_l2_path'] + r['beta_fname']
                        active_img_cleaned = get_beta_img(beta_img_filepath)
                        if active_img_cleaned is None:
                            continue

                        print("img mean is " + str(np.mean(active_img_cleaned.get_fdata()[np.isnan(active_img_cleaned.get_fdata())==False])))


                        if spatial_mean_center:
                            #mean center the nifti image
                            raise NotImplementedError("spatial mean centering not implemented")
                            active_img_cleaned = active_img_cleaned - active_img_cleaned.mean()


                            #active_img_cleaned = active_img_cleaned - active_img_cleaned.mean()

                        #for each mask
                        roi_data_from_masks = get_roi_data_for_beta(
                                r['subject_id'],
                                r['wave'],
                                r['spm_l2_path'],
                                condition,
                                r['beta_fname'],
                                raw_mask_list, mask_df, active_img_cleaned)
                        roi_data_from_masks_df = pd.DataFrame(roi_data_from_masks)
                        roi_data_from_masks_df['run'] = r['task_run']
                        roi_data_all.append(roi_data_from_masks_df)
                        
        roi_data_all_df = pd.concat(roi_data_all)

        if collapse_runs:
            roi_data_all_df = collapse_runs(roi_data_all_df)

        return roi_data_all_df
    


    def get_roi_data_for_l2_contrasts(self, beta_list, condition_list,roi_df):
        col_function=lambda img_name: "contrast_" + img_name + "_fname"
        #iterate through each mask
        return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function))

    def subtract_mean(self, image_series):
        img_mask = self.brain_mask_path
                # Get a mean image within the roi
        mean_image = nil.image.math_img("np.mean(img1 * img2, axis=-1)",
                                img1=image_series, img2=img_mask)
        
        # duplicate the mean image to have the same shape as image_series
        mean_image = nil.image.new_img_like(image_series, 
                                        np.repeat(mean_image.get_fdata()[..., np.newaxis], 
                                                image_series.shape[-1], axis=-1))

        # Subtract the mean from each image
        standardized_images = nil.image.math_img('(img1 - img2)', img1=image_series, img2=mean_image)
        return(standardized_images)
    
    def get_roi_data_across_all_betas(self,beta_list: pd.DataFrame, condition,col_function, raw_roi_list, roi_df):

        colname = col_function(condition)
    
        beta_filepaths = [r['spm_l2_path'] + r[colname] for i, r in beta_list.iterrows()]
        #strip out the filepaths that don't exist
        filepath_exists = [os.path.exists(b) for b in beta_filepaths]
        beta_list_exists = beta_list[filepath_exists].reset_index(drop=True,inplace=False)
        del beta_list

        #get just the beta_filepaths that exist by indexing beta_filepaths with the boolean array
        beta_filepath_exists = [beta_filepaths[i] for i in range(len(beta_filepaths)) if filepath_exists[i]]

        print("loading " + str(len(beta_filepath_exists)) + "images...")
        image_series = nil.image.load_img(beta_filepath_exists)
        print("...loaded.")

        #clean the image
        image_series_cleaned = nil.image.clean_img(image_series,detrend=self.center_data ,standardize=self.scale_data)

        if self.image_standardize:
            #get the f
            image_series_cleaned = self.subtract_mean(image_series_cleaned)

        #image_series_cleaned.to_filename('cleaned_img.nii.gz')
        # img_cleaned_fdata = image_series_cleaned.get_fdata()
        # #get the mean over the image series
        # img_mean = nil.image.mean_img(image_series_cleaned)
        # #plot
        # nil.plotting.plot_img(img_mean)

        roi_data_for_beta = self.get_roi_data_for_beta(
                    beta_list_exists['subject_id'].tolist(),
                    beta_list_exists['wave'].tolist(),
                    beta_list_exists['spm_l2_path'].tolist(),
                    condition,
                    beta_list_exists[colname].tolist(),
                    raw_roi_list, roi_df, image_series_cleaned)
        
        roi_data_all = roi_data_for_beta

        return(roi_data_all)




    def get_roi_data_for_roi_col(self,beta_list, condition,col_function, raw_roi_list, roi_df):

        colname = col_function(condition)

        roi_data_all = []

        #iterate through each image, if it exists, and pull the ROI
        for i, r in beta_list.iterrows():
            #print(r['subject_id'] + ", " + str(r['wave']))xx
            #print(r.index)
            #some checks where skipping is necessary
            if colname not in beta_list.columns:
                continue
            if type(r[colname])==float:
                if np.isnan(r[colname]):
                    continue
                #we want an error thrown if somehow it's a float but not nan,
                # so we handle that case here.
            if r[colname] is None:
                continue
            
        
            #get the beta image from which to pull an ROI
            beta_img_filepath = r['spm_l2_path'] + r[colname]
            active_img_cleaned = get_beta_img(beta_img_filepath)

            if self.center_data or self.scale_data:
                raise NotImplementedError("centering and scaling not implemented")

            if active_img_cleaned is None:
                continue

            #for each mask
            roi_data_for_beta = self.get_roi_data_for_beta(
                    r['subject_id'],
                    r['wave'],
                    r['spm_l2_path'],
                    condition,
                    r[colname],
                    raw_roi_list, roi_df, active_img_cleaned)
            
            roi_data_all.append(pd.DataFrame(roi_data_for_beta))
            
        return(roi_data_all)
    
    def get_roi_data_for_l2(self, beta_list,condition_list,roi_df,col_function):
        raw_roi_list = self.load_rois(roi_df)
        #now iterate through each beta image
        #for each beta type
        roi_data_all = []
        for condition in condition_list:
            print(condition)
            
            if self.load_all_images_simultaneously:
                roi_data = self.get_roi_data_across_all_betas(beta_list, condition,col_function, raw_roi_list, roi_df)
            else:
                roi_data = self.get_roi_data_for_roi_col(beta_list, condition,col_function, raw_roi_list)
            roi_data_all = roi_data_all + roi_data
            

        roi_data_df = pd.concat(roi_data_all)
        return roi_data_df


    def get_roi_data_for_l2_betas(self, beta_list, condition_list,roi_df):
        col_function=lambda img_name: "beta_" + img_name + "_fname"
        #iterate through each mask
        return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function))




